In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib
from scipy.ndimage import gaussian_filter
from astropy.io import fits
from astropy.utils.data import download_file
from astropy.modeling import models, fitting
import math

In [ ]:
imagefile = fits.open('mosaic.fits')
header = fits.getheader('mosaic.fits',0)
data = imagefile[0].data
height, width = data.shape #Defining dimensions of data grid

In [ ]:
def calculate_magnitude(data, xc, yc, radius):
    #Defining aperture 
    x_min = np.clip(xc - radius, a_min=0, a_max=width)
    x_max = np.clip(xc + radius, a_min=0, a_max=width)
    y_min = np.clip(yc - radius, a_min=0, a_max=height)
    y_max = np.clip(yc + radius, a_min=0, a_max=height)
    #Extracting aperture data
    slice_region = data[y_min:y_max + 1, x_min:x_max + 1]
    
    if slice_region.size == 0:
        raise ValueError(f"Empty box extracted at coordinates ({xc}, {yc}).")
    #Ensuring that the central pixel is the brightest 
    if slice_region[(radius, radius)] != slice_region.max():
        return None, None, None, None, None, None #return no parameters

    y_indices, x_indices = np.indices(slice_region.shape)# Coordinates of all pixels within aperture
    y_center, x_center = radius, radius  # Center of aperture
    distances = np.sqrt((x_indices - x_center) ** 2 + (y_indices - y_center) ** 2) # calculating the distance of each pixel from the center

    # Separate aperture and background pixels
    aperture_pixels = slice_region[distances <= radius]
    background_pixels = slice_region[distances > radius]

    if aperture_pixels.size == 0 or background_pixels.size == 0: #Helps avoid apertures of background noise/bleeding
        return None, None, None, None, None, None

    if aperture_pixels.size == 0 or background_pixels.size == 0:
        raise ValueError(
            f"Invalid aperture or background at coordinates ({xc}, {yc}).")

    total_counts = np.sum(slice_region)#Total number of pixels
    # Mean flux of background
    background_mean = np.mean(background_pixels)

    # Total flux in aperture
    aperture_flux = np.sum(aperture_pixels)

    #finding errors
    bg_std_dev = np.std(background_pixels)  # Standard deviation of background pixels
    bg_error = bg_std_dev / np.sqrt(background_pixels.size)  # Error in mean background


    # Subtract background contribution from the total flux
    source_flux = aperture_flux - (background_mean * aperture_pixels.size) 

    # Calculate magnitude
    magnitude = 25.3 - (2.5 * np.log10(max(source_flux, 1e-10)))#Zero point is 25.3 from header
    #magnitude error is err(zpt) + 0.434 (bg_error/source flux)

    zpt_error = 2E-2 # From header, error in zero point

    flux_error = bg_error * aperture_pixels.size #Error in flux

    magnitude_error = zpt_error + 0.434 * (bg_error/source_flux)  # 0.434 = 1/ln 10 

    return total_counts, aperture_flux, source_flux, flux_error, magnitude, magnitude_error #Parameters to be catalogued

In [ ]:
#Simulated data used to test accuracy of method
def generate_zeros_image(height, width): 
    return np.zeros((height, width))

def generate_noise_image(height, width):
    # Random values between 3300 and 10000
    return np.random.randint(3300, 10000, size = (height, width))


def generate_gaussian_image(height, width, amplitude):
    # Create an empty image
    image = np.zeros((height, width))

    # Define the center and amplitude of the gaussian source
    xc, yc = width // 2, height // 2  # Center of the image
    print(xc, yc)
    #add gaussian to empty image
    image[yc, xc] = amplitude
    image = gaussian_filter(image, sigma=1.0)  # Smooth with a Gaussian kernel

    return image



In [ ]:
#Definition to catalogue and recognise sources
def mag_plot(data, cfile, radius): #cfile is catalogue file
    number = 0

    with open(cfile, "w") as file:

        file.write("x_coord\ty_coord\tTotal_counts\tAperture_flux\tNet_flux\tFlux_Error\tMagnitude\tMag_Error\n")#Write header of catalogue

        for i in range(data.size):

            max_index = np.argmax(data)#Find brightest unprocessed pixel of image

            coordinates = np.unravel_index(max_index, data.shape)
            max_value = data[coordinates]
            #print(max_value)

            if max_value < 3400: #3400 is roughly the background flux
                break

            xc = coordinates[1]#defining central coordinates
            yc = coordinates[0]

            #Defining an aperture around source, same process as in 'calculate_magnitude'
            x_min = np.clip(xc - radius, a_min=0, a_max=width)
            x_max = np.clip(xc + radius, a_min=0, a_max=width)
            y_min = np.clip(yc - radius, a_min=0, a_max=height)
            y_max = np.clip(yc + radius, a_min=0, a_max=height)

            slice = data[y_min:y_max+1, x_min:x_max+1] #array slicing excludes the last index hence the +1

                 
            if slice.size == 0:
                raise ValueError("The extracted box is empty.")

            #Empty arrays to be filled with data
            dists = []
            intensities = []
            log_intensities = []

            #new relative coordinates of centre
            xc_slice = xc - x_min
            yc_slice = yc - y_min

            #Calculates distance from center as function of intensites to create radial intensity profile
            for i in range(slice.size):
                coords = np.unravel_index(i, slice.shape)
                intensity = slice[coords]
                # coords[0] is relative y coordinate, coords[1] is relative x coordinate
                distance = np.sqrt((coords[1] - xc_slice) ** 2 + (coords[0] - yc_slice) ** 2)
                dists.append(distance)
                intensities.append(intensity)
                log_intensities.append(np.log10(max(intensity, 1E-10)))

            r = (np.corrcoef(dists, intensities))[0, 1]#Needs a roughly linear correlation

            onecount = 0
            percentile_98 = np.percentile(slice.flatten(), 98)#Only 98th percentile of image
            num_pixels_above = np.sum(slice > percentile_98)
            gradient = np.gradient(intensities, dists)


            for i in np.nditer(slice):
                if i == 1:
                    onecount = onecount+1
            #Checking correlatin and smoothness of gradient
            if r <= -0.4 and onecount <= radius and abs(gradient[1]) < 100:

                total_counts, aperture_flux, source_flux, flux_error, magnitude, magnitude_error = calculate_magnitude(data, xc, yc, radius)

                if magnitude == None:
                    pass
                else:
                    file.write(f"{xc:.2f}\t{yc:.2f}\t{total_counts:.2f}\t{aperture_flux:.2f}\t"
                           f"{source_flux:.2f}\t{flux_error:.2f}\t{magnitude:.2f}\t{magnitude_error:.2f}\n")
            #Get total count 
                number = number+1

                print(number)


            data[y_min:y_max + 1, x_min:x_max + 1] = 1 #Masks data that has already been processed 

    return(number)

In [ ]:
#Testing simulated data
zero_image = generate_zeros_image(100,100)
cfile_zero = "test_zero_catalogue.txt"
mag_plot(zero_image,cfile_zero, 4)

random_image = generate_noise_image(100,100)
cfile_noise = "test_noise_catalogue.txt"
mag_plot(random_image, cfile_noise, 4)

gaussian_source = generate_gaussian_image(100,100, 4000)
cfile_gaussian = "test_gaussian_catalogue.txt"
mag_plot(gaussian_source, cfile_gaussian, 4)

In [ ]:
#Final catalogue with radius = 4
cfile = "radius4_catalogue.txt"

mag_plot(data, cfile, 4)
